Weather prevision for top 15 most populous municipalities in Spain (with storing of each municipality in cloud storage)

In [1]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json"

In [2]:
from google.cloud import storage

In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import json

In [4]:
url = 'https://es.wikipedia.org/wiki/Anexo:Municipios_de_Espa%C3%B1a_por_poblaci%C3%B3n'
headers = {"User-Agent": "Mozilla/5.0"}

In [5]:
req = Request(url, headers=headers)
raw_web = urlopen(req).read()
soup = BeautifulSoup(raw_web, 'html.parser')

In [6]:
def save_gcloud(_data, _name, _ruta):
    
    client = storage.Client()
    bucket = client.bucket('imf-nov-2022')
    
    blob = bucket.blob(_ruta + _name)
    blob.upload_from_string(_data)
    
    print('Uploaded Successfully')    
    
def getfileslist(_carpeta):
    
    client = storage.Client()
    blobs = client.list_blobs('imf-nov-2022', prefix = _carpeta)
    
    lista = []
    
    for blob in blobs:
        
        lista.append(blob.name)
        
    return lista

def getfile(_blob):
    
    client = storage.Client()
    bucket = client.bucket('imf-nov-2022')
    
    blob = bucket.blob(_blob)
    content = blob.download_as_string()
    
    return content

def txt_2_json(_content):
    
    return json.loads(_content)
    

In [7]:
tbodies = soup.find_all("tbody")

In [8]:
rows = []

for tbody in tbodies:
    for tr in tbody.find_all('tr')[1:]:
        row = [td.text.strip() for td in tr.find_all('td')]
        rows.append(row)

In [9]:
rows

[['1',
  'Madrid',
  '3 280 782',
  'Madrid\xa0Madrid',
  'Comunidad de Madrid\xa0Comunidad de Madrid'],
 ['2',
  'Barcelona',
  '1 636 193',
  'Barcelona\xa0Barcelona',
  'Cataluña\xa0Cataluña'],
 ['3',
  'Valencia',
  '792 492',
  'ValenciaValencia',
  'Comunidad\xa0Valenciana\xa0Comunidad\xa0Valenciana'],
 ['4', 'Sevilla', '681 998', 'Sevilla\xa0Sevilla', 'Andalucía\xa0Andalucía'],
 ['5', 'Zaragoza', '673 010', 'Zaragoza\xa0Zaragoza', 'Aragón\xa0Aragón'],
 ['6', 'Málaga', '579 076', 'Málaga\xa0Málaga', 'Andalucía\xa0Andalucía'],
 ['7', 'Murcia', '462 979', 'Región de Murcia\xa0Región de Murcia'],
 ['8', 'Palma', '415 940', 'Islas Baleares\xa0Islas Baleares'],
 ['9',
  'Las Palmas de Gran Canaria',
  '378 797',
  'Las PalmasLas Palmas',
  'Canarias'],
 ['10', 'Bilbao', '344 127', 'Vizcaya\xa0Vizcaya', 'País Vasco\xa0País Vasco'],
 ['11',
  'Alicante',
  '338 577',
  'AlicanteAlicante',
  'Comunidad\xa0Valenciana\xa0Comunidad\xa0Valenciana'],
 ['12', 'Córdoba', '319 515', 'Córdoba\xa0

In [10]:
data = []

for row in rows:
    
    if len(row) == 5:
        Id=int(row[0])
        nombre=row[1]
        poblacion= int(row[2].replace(" ", ""))
        provincia= row[3].split("\xa0")[0].strip()
        comunidad_autonoma= row[4].split("\xa0")[0].strip()
        dic = {'id': Id, 'nombre': nombre, 'poblacion':poblacion, 'provincia': provincia, 'comunidad_autonoma': comunidad_autonoma}
        data.append(dic)
    elif len(row) == 4:
        Id=int(row[0])
        nombre=row[1]
        poblacion= int(row[2].replace(" ", ""))
        provincia= row[3].split("\xa0")[0].strip()
        comunidad_autonoma= row[3].split("\xa0")[0].strip()
        dic = {'id': Id, 'nombre': nombre, 'poblacion':poblacion, 'provincia': provincia, 'comunidad_autonoma': comunidad_autonoma}
        data.append(dic)


In [11]:
data

[{'id': 1,
  'nombre': 'Madrid',
  'poblacion': 3280782,
  'provincia': 'Madrid',
  'comunidad_autonoma': 'Comunidad de Madrid'},
 {'id': 2,
  'nombre': 'Barcelona',
  'poblacion': 1636193,
  'provincia': 'Barcelona',
  'comunidad_autonoma': 'Cataluña'},
 {'id': 3,
  'nombre': 'Valencia',
  'poblacion': 792492,
  'provincia': 'ValenciaValencia',
  'comunidad_autonoma': 'Comunidad'},
 {'id': 4,
  'nombre': 'Sevilla',
  'poblacion': 681998,
  'provincia': 'Sevilla',
  'comunidad_autonoma': 'Andalucía'},
 {'id': 5,
  'nombre': 'Zaragoza',
  'poblacion': 673010,
  'provincia': 'Zaragoza',
  'comunidad_autonoma': 'Aragón'},
 {'id': 6,
  'nombre': 'Málaga',
  'poblacion': 579076,
  'provincia': 'Málaga',
  'comunidad_autonoma': 'Andalucía'},
 {'id': 7,
  'nombre': 'Murcia',
  'poblacion': 462979,
  'provincia': 'Región de Murcia',
  'comunidad_autonoma': 'Región de Murcia'},
 {'id': 8,
  'nombre': 'Palma',
  'poblacion': 415940,
  'provincia': 'Islas Baleares',
  'comunidad_autonoma': 'Islas

In [ ]:
carpeta = 'municipios/'

In [ ]:
for file in data:
    data_json = json.dumps(file)
    city = file['nombre'] + '.json'
    save_gcloud(data_json, city, carpeta)

In [4]:
import mysql.connector
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [5]:
gc = mysql.connector.connect(
    host= '34.175.128.194',
    user= 'root',
    password= '12345',
)

In [12]:
municipios = []

for mun in getfileslist('municipios/'):

    municipios.append(txt_2_json(getfile(mun)))
    
municipios

[{'id': 947,
  'nombre': 'Abadiano',
  'poblacion': 7708,
  'provincia': 'Vizcaya',
  'comunidad_autonoma': 'País Vasco'},
 {'id': 1122,
  'nombre': 'Abanilla',
  'poblacion': 6146,
  'provincia': 'Región de Murcia',
  'comunidad_autonoma': 'Región de Murcia'},
 {'id': 814,
  'nombre': 'Abanto y Ciérvana',
  'poblacion': 9369,
  'provincia': 'Vizcaya',
  'comunidad_autonoma': 'País Vasco'},
 {'id': 611,
  'nombre': 'Abarán',
  'poblacion': 12992,
  'provincia': 'Región de Murcia',
  'comunidad_autonoma': 'Región de Murcia'},
 {'id': 1215,
  'nombre': 'Abegondo',
  'poblacion': 5534,
  'provincia': 'La CoruñaLa Coruña',
  'comunidad_autonoma': 'Galicia'},
 {'id': 625,
  'nombre': 'Abrera',
  'poblacion': 12697,
  'provincia': 'Barcelona',
  'comunidad_autonoma': 'Cataluña'},
 {'id': 1223,
  'nombre': 'Aceuchal',
  'poblacion': 5456,
  'provincia': 'Badajoz',
  'comunidad_autonoma': 'Extremadura'},
 {'id': 152,
  'nombre': 'Adeje',
  'poblacion': 49270,
  'provincia': 'Santa Cruz de Tene

In [6]:
def sql(_query):
    return pd.read_sql_query(_query, gc)

In [7]:
cursor = gc.cursor()

In [ ]:
cursor.execute("""CREATE TABLE demo.municipios (
                Id int NOT NULL,
                nombre varchar(100),
                poblacion int,
                provincia varchar(150),
                comunidad_autonoma varchar(150),
                PRIMARY KEY (Id))
                """)

In [ ]:
gc.commit()

In [13]:
def fix_duplicate_ids(municipios):
    id_count = {}
    for m in municipios:
        id = m['id']
        if id in id_count:
            id_count[id] += 1
            m['id'] = len(municipios) + 1
        else:
            id_count[id] = 1
    return municipios

In [14]:
data = fix_duplicate_ids(municipios)

In [15]:
data

[{'id': 947,
  'nombre': 'Abadiano',
  'poblacion': 7708,
  'provincia': 'Vizcaya',
  'comunidad_autonoma': 'País Vasco'},
 {'id': 1122,
  'nombre': 'Abanilla',
  'poblacion': 6146,
  'provincia': 'Región de Murcia',
  'comunidad_autonoma': 'Región de Murcia'},
 {'id': 814,
  'nombre': 'Abanto y Ciérvana',
  'poblacion': 9369,
  'provincia': 'Vizcaya',
  'comunidad_autonoma': 'País Vasco'},
 {'id': 611,
  'nombre': 'Abarán',
  'poblacion': 12992,
  'provincia': 'Región de Murcia',
  'comunidad_autonoma': 'Región de Murcia'},
 {'id': 1215,
  'nombre': 'Abegondo',
  'poblacion': 5534,
  'provincia': 'La CoruñaLa Coruña',
  'comunidad_autonoma': 'Galicia'},
 {'id': 625,
  'nombre': 'Abrera',
  'poblacion': 12697,
  'provincia': 'Barcelona',
  'comunidad_autonoma': 'Cataluña'},
 {'id': 1223,
  'nombre': 'Aceuchal',
  'poblacion': 5456,
  'provincia': 'Badajoz',
  'comunidad_autonoma': 'Extremadura'},
 {'id': 152,
  'nombre': 'Adeje',
  'poblacion': 49270,
  'provincia': 'Santa Cruz de Tene

In [ ]:
for m in range(len(data)):
    
    Id = data[m]['id']
    nombre = data[m]['nombre']
    poblacion = data[m]['poblacion']
    provincia = data[m]['provincia']
    comunidad_autonoma = data[m]['comunidad_autonoma']
    cursor.execute(f"""INSERT INTO demo.municipios (Id, nombre, poblacion, provincia, comunidad_autonoma) 
               VALUES ({Id}, '{nombre}', {poblacion}, '{provincia}', '{comunidad_autonoma}')
               """)
    gc.commit()

In [8]:
sql("SELECT * FROM demo.municipios")

,Id,nombre,poblacion,provincia,comunidad_autonoma
0,1,Madrid,3280782,Madrid,Comunidad de Madrid
1,2,Barcelona,1636193,Barcelona,Cataluña
2,3,Valencia,792492,ValenciaValencia,Comunidad
3,4,Sevilla,681998,Sevilla,Andalucía
4,5,Zaragoza,673010,Zaragoza,Aragón
...,...,...,...,...,...
1309,1311,Vedra,5032,La CoruñaLa Coruña,Galicia
1310,1312,Cerceda,5019,La CoruñaLa Coruña,Galicia
1311,1313,Pioz,5013,Guadalajara,Castilla-La Mancha
1312,1314,Ribadavia,5003,Orense,Galicia


In [9]:
top_15 = sql("SELECT * FROM demo.municipios ORDER BY poblacion DESC LIMIT 15")

In [10]:
top_15

,Id,nombre,poblacion,provincia,comunidad_autonoma
0,1,Madrid,3280782,Madrid,Comunidad de Madrid
1,2,Barcelona,1636193,Barcelona,Cataluña
2,3,Valencia,792492,ValenciaValencia,Comunidad
3,4,Sevilla,681998,Sevilla,Andalucía
4,5,Zaragoza,673010,Zaragoza,Aragón
5,6,Málaga,579076,Málaga,Andalucía
6,7,Murcia,462979,Región de Murcia,Región de Murcia
7,8,Palma,415940,Islas Baleares,Islas Baleares
8,9,Las Palmas de Gran Canaria,378797,Las PalmasLas Palmas,Canarias
9,10,Bilbao,344127,Vizcaya,País Vasco


In [11]:
top_15['nombre']

0                         Madrid
1                      Barcelona
2                       Valencia
3                        Sevilla
4                       Zaragoza
5                         Málaga
6                         Murcia
7                          Palma
8     Las Palmas de Gran Canaria
9                         Bilbao
10                      Alicante
11                       Córdoba
12                    Valladolid
13                          Vigo
14                         Gijón
Name: nombre, dtype: object

In [12]:
from cities_coordinates import CityCoordinator
import requests as rq
c=CityCoordinator()

In [13]:
def getweather(city):
    
    BASE = "https://api.open-meteo.com/v1/forecast/"
    coord = c.get_city(city_name=city.title(),country_code_iso="ES")
    
    lon = coord["location"]["lon"]
    lat = coord["location"]["lat"]
    
    params= {
        "latitude": lat,
        "longitude":lon,
        "daily":["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
        "timezone":"GMT"
    }

    response = rq.get(BASE,params=params)
    db = response.json()
    db2 = {
          "time": db['daily']['time'],
          "temperature_2m_max": db['daily']['temperature_2m_max'],
          "temperature_2m_min": db['daily']['temperature_2m_min'],
          "precipitation_sum": db['daily']['precipitation_sum']
           }
    return db2
    # return pd.DataFrame(db2, columns = db2.keys())

In [14]:
weather = []

for city in top_15['nombre']:
    
    w = {city: getweather(city) for city in top_15['nombre']}
    
    weather.append(w)

In [15]:
weather

[{'Madrid': {'time': ['2023-03-13',
    '2023-03-14',
    '2023-03-15',
    '2023-03-16',
    '2023-03-17',
    '2023-03-18',
    '2023-03-19'],
   'temperature_2m_max': [20.1, 17.2, 18.6, 22.5, 14.2, 15.4, 19.1],
   'temperature_2m_min': [11.1, 7.7, 5.9, 6.9, 7.9, 6.3, 7.1],
   'precipitation_sum': [0.1, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0]},
  'Barcelona': {'time': ['2023-03-13',
    '2023-03-14',
    '2023-03-15',
    '2023-03-16',
    '2023-03-17',
    '2023-03-18',
    '2023-03-19'],
   'temperature_2m_max': [20.6, 21.8, 15.9, 14.1, 16.7, 17.4, 20.6],
   'temperature_2m_min': [11.6, 13.4, 11.0, 8.9, 8.4, 10.5, 9.3],
   'precipitation_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]},
  'Valencia': {'time': ['2023-03-13',
    '2023-03-14',
    '2023-03-15',
    '2023-03-16',
    '2023-03-17',
    '2023-03-18',
    '2023-03-19'],
   'temperature_2m_max': [31.1, 24.5, 22.9, 17.3, 18.7, 20.4, 20.0],
   'temperature_2m_min': [14.9, 15.2, 13.0, 10.6, 10.5, 10.8, 10.1],
   'precipitation_sum': [0.0, 0

In [71]:
def prevision():
    
    dfs = []
    
    for i in weather[0]:
        
        df = pd.DataFrame(weather[0][i])
        dfs.append(df)
    
    return pd.concat(dfs)



In [79]:
final = prevision()

In [80]:
names = list(weather[0].keys())
names = [i for i in names for j in range(7)]
final.insert(loc=0, column= 'Municipio', value=names)

In [81]:
final

,Municipio,time,temperature_2m_max,temperature_2m_min,precipitation_sum
0,Madrid,2023-03-13,20.1,11.1,0.1
1,Madrid,2023-03-14,17.2,7.7,0.0
2,Madrid,2023-03-15,18.6,5.9,0.0
3,Madrid,2023-03-16,22.5,6.9,0.0
4,Madrid,2023-03-17,14.2,7.9,3.0
...,...,...,...,...,...
2,Gijón,2023-03-15,18.4,8.9,0.0
3,Gijón,2023-03-16,22.5,10.2,0.0
4,Gijón,2023-03-17,16.9,11.8,1.3
5,Gijón,2023-03-18,16.8,10.2,0.0
